# HED: Putting Vietnamese food on the map

### SUBHED: New York restaurants catch the eye of the Michelin guide

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

/Users/onlyandrewn/.pyenv/versions/3.10.3/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [2]:
# !pip install bs4

In [3]:
response = requests.get("https://guide.michelin.com/us/en/restaurants/vietnamese")
doc = BeautifulSoup(response.text)
doc

<!DOCTYPE html>
<html class="full-screen-mobile" lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, user-scalable=0" name="viewport"/>
<meta content="" name="author"/>
<meta content="#bd2333" name="theme-color"/>
<meta content="vFUp7XI3G-xKLevkT2qftuscMInOxirdCu0oYr7_uLM" name="google-site-verification"/>
<meta content="eZwPBJ-0r-9qysyJy_aEPti-m_vr-h_oUJy2qUR1144" name="google-site-verification"/>
<meta content="michelin_revamp" name="layout"/>
<meta content="Article" property="og:type"/>
<meta content="MICHELIN Guide" property="og:site_name"/>
<meta content="https://guide.michelin.com/us/en/restaurants/vietnamese" property="og:url"/>
<meta content="" itemprop="name"/>
<meta content="" property="og:title"/>
<title></title>
<meta content="" itemprop="description"/>
<meta content="" property="og:description"/>
<meta content="" name="description"/>
<link href="https://guide.michelin.com/us/en/restaurants/vietnamese" rel="canonical"/>
<link h

In [4]:
!pip install Unidecode

You should consider upgrading via the '/Users/onlyandrewn/.pyenv/versions/3.10.3/bin/python3.10 -m pip install --upgrade pip' command.


In [5]:
import re
from unidecode import unidecode

In [6]:
pages_start = 1
pages_total = 3

pages = []
restaurants = []

base_url = "https://guide.michelin.com"

# https://guide.michelin.com/us/en/restaurants/vietnamese
# https://guide.michelin.com/us/en/search?type=restaurant&page=1&q=vietnamese
# https://guide.michelin.com/us/en/search?type=restaurant&page=2&q=vietnamese
# https://guide.michelin.com/us/en/search?type=restaurant&page=3&q=vietnamese

while pages_start <= pages_total:
    
    response = requests.get("https://guide.michelin.com/us/en/search?type=restaurant&page=" + str(pages_start) + "&q=vietnamese")
    doc = BeautifulSoup(response.text)
    doc
    
    for restaurant in doc.find_all("div", class_="js-restaurant__list_item"):
        name = (restaurant.find("h3", class_="card__menu-content--title").find("a").text).strip()
        slug = name.lower() \
            .replace(" ","-") \
            .replace("---","-") \
            .replace("(","") \
            .replace(")","")

        slug = unidecode(slug)

        url = base_url + restaurant.find("h3").a.get("href")   
        image = restaurant.find("a", class_="image-wrapper").find("img")["src"]
        # TODO - DOWNLOAD ALL THE IMAGES, BUT SAVE THEM AS THE SLUG OF THE RESTAURANT.JPG

        guide = restaurant.find("div", class_="card__menu-content--rating").find("span").text.strip()
        city = (restaurant.find("div", class_="card__menu-footer--location").text).strip()
        lat = restaurant["data-lat"]
        lon = restaurant["data-lng"]
        category = (restaurant.find("div", class_="card__menu-footer--price").text).strip()

        if category == "Vietnamese, Vietnamese":
            category = "Vietnamese"
        elif category == "Fusion, Fusion":
            category = "Fusion"
        elif category == "South East Asian, South East Asian":
            category = "South East Asian"
        elif category == "Contemporary, Contemporary":
            category = "Contemporary"
        elif category == "Asian and Western, Asian and Western":
            category = "Asian and Western"
        elif category == "Indonesian, Indonesian":
            category = "Indonesian"

        restaurant = {
            "name": name,
            "slug": slug,
            "url": url,
            "image": image,
            "guide": guide,
            "city": city,
            "lat": lat,
            "lon": lon,
            "category": category
        }
    
        restaurants.append(restaurant)
        
    pages_start = pages_start + 1

df = pd.DataFrame(restaurants)
df.to_csv("restaurants.csv", index=False)

In [7]:
# NEW YORK LEADS ALL CITIES FOR VIETNAMESE RESTAURANTS RECOGNIZED BY MICHELIN
# NO VIETNAMESE RESTAURANT CURRENTLY HAS A SINGLE MICHELIN STAR
# LA MAISON 1888, A FRENCH RESTAURANT IN DA NANG AT ONE POINT HAD A MICHELIN STAR

df["city"].value_counts()

New York                    5
Bangkok                     3
Chicago                     3
Tokyo                       3
Tampa                       2
Berlin                      2
Washington                  2
Garden Grove                2
San Leandro                 1
Palo Alto                   1
Huntington Beach            1
Kyoto                       1
Milan                       1
Hong Kong                   1
Phra Nakhon Si Ayutthaya    1
Orlando                     1
Glasgow City                1
Paris                       1
Taichung                    1
Antwerpen                   1
Woluwe-Saint-Lambert        1
Miami                       1
Oceanside                   1
Name: city, dtype: int64

In [8]:
df[df["guide"] == "MICHELIN Guide USA"]["city"].value_counts()

New York            5
Chicago             3
Tampa               2
Washington          2
Garden Grove        2
Orlando             1
Miami               1
Huntington Beach    1
Palo Alto           1
San Leandro         1
Oceanside           1
Name: city, dtype: int64

In [9]:
df[df["guide"] == "MICHELIN Guide USA"]["city"].value_counts().sum()

20

In [10]:
# 54% OF VIETNAMESE RESTAURANTS ARE IN THE USA
(df[df["guide"] == "MICHELIN Guide USA"]["city"].value_counts().sum() / df["guide"].value_counts().sum())* 100

54.054054054054056

In [11]:
response = requests.get("https://guide.michelin.com/us/en/search?type=restaurant&page=2&q=vietnamese")
doc = BeautifulSoup(response.text)
# doc

In [12]:
urls = df["url"]

len(urls)

37

In [13]:
restaurant_info = []

for url in urls:
    response = requests.get(url)
    doc = BeautifulSoup(response.text)
    doc
    
    for data in doc.find_all("body"):        
        name = data.find("h2", class_="restaurant-details__heading--title").text
        address = data.find("ul", class_="restaurant-details__heading--list").find("li").text
        price = data.find("li", class_="restaurant-details__heading-price").text \
            .replace(" ", "") \
            .replace("\n", "") \
            .replace("USD•Vietnamese", " USD") \
            .replace("EUR•Vietnamese,Asian", " EUR") \
            .replace("EUR•Vietnamese", " EUR") \
            .replace("GBP•Vietnamese", " GBP") \
            .replace("HKD•Vietnamese", " HKD") \
            .replace("TWD•Vietnamese", " TWD") \
            .replace("JPY•Vietnamese", " JPY") \
            .replace("Vietnamese,SouthEastAsian", "") \
            .replace(",Contemporary", "") \
            .replace(",Noodles", "") \
            .replace(",ModernCuisine", "") \
            .replace(",French", "") \
            .replace("USD•Fusion,Vietnamese", " USD") \
            .replace("EUR•SouthEastAsian,Vietnamese", " EUR") \
            .replace("USD•Contemporary,Vietnamese", " USD") \
            .replace("EUR•AsianandWestern,VietnameseContemporary", " EUR") \
            .replace("USD•Indonesian,ThaiandVietnamese", " USD") \
            .replace("USD,SouthEastAsian", " USD") \
            .replace("THB•Vietnamese", " THB") \
            .strip()
        desc = data.find("div", class_="js-show-description-text").text
        
        if data.find("ul", class_="restaurant-details__classification--list"):
            award = data.find("ul", class_="restaurant-details__classification--list").text \
                .replace("=", "") \
                .replace("\n", "") \
                .strip() \
                .replace("                                2                                    Small Shop", "; Small Shop")
        else:
            award = "None"
        
        if data.find("div", class_="collapse__block-title d-flex"):
            url = data.find("div", class_="collapse__block-title d-flex").find("span").text
        else:
            url = "None"

        info = {
            "name": name,
            "address": address,
            "price": price,
            "desc": desc,
            "award": award,
            "url": url
        }
        
        restaurant_info.append(info)
        
df = pd.DataFrame(restaurant_info)
df.to_csv("restaurant_info.csv", index=False)

In [14]:
df[df["address"].str.contains("New York")]

,name,address,price,desc,award,url
5,Falansai,"112 Harrison Pl., New York, 11237, USA",25-50 USD,\nOccupying the same corner location since 201...,None,https://www.falansai.com/
28,Đi Ăn Đi,"68 Greenpoint Ave., New York, 11222, USA",44-50 USD,"\nBrought to you by Dennis Ngo, Kim Hoang and ...",None,https://www.diandi.nyc/
29,Thái So´n,"89 Baxter St., New York, 10013, USA",25 USD,\nThái So´n is by far the best of the bunch in...,None,None
30,Van Đa,"234 E. 4th St., New York, 10009, USA",50-55 USD,\nRestaurateurs Yen Ngo and Chef Hannah Wong a...,"Bib Gourmand: good quality, good value cooking",https://www.vanda.nyc/
31,Bolero,"177 Bedford Ave., New York, 11211, USA",50 USD,\nThis gem stands out for its keen understandi...,"Bib Gourmand: good quality, good value cooking",https://bolero.nyc/


In [15]:
### TODO - FIX CUISINE_LIST AND CUISINE_NAME VARIABLES

cuisines = []

response = requests.get("https://guide.michelin.com/en/restaurants")
doc = BeautifulSoup(response.text)
# doc

for cuisine in doc.find("div", id="cuisine").find_all("label"):    
    cuisine_list = cuisine.text \
        .replace(" ", "") \
        .replace("\n", "")

    cuisine_name = re.sub(".\d+.", "", cuisine_list)
    
    cuisine_info = {
        "full": cuisine_list,
        "name": cuisine_name
    }
    
    cuisines.append(cuisine_info)

In [16]:
df = pd.DataFrame(cuisines)

df["num"] = df["full"].str.extract("(\d+)")
df["num"] = df["num"].astype(int)

In [17]:
# 15,930 RESTAURANTS EXIST IN THE CURRENT VERSION OF THE MICHELIN GUIDE
# WHEN PRESENTED AS CATEGORIES, THERE ARE 15987, LIKELY DUE TO RESTAURANTS APPEARING IN MORE THAN 1 CATEGORY
df["num"].sum()

15965

In [18]:
df.to_csv("cuisines.csv", index=False)

# MENUS

In [19]:
import camelot

### Falansai

In [20]:
# !pip install pdfminer.six

In [21]:
### WRONG MENU

# restaurant = "Falansai"

# menus = [
#     {"name": "DINNER MENU-TOGO.pdf", "type": "dinner"}, 
#     {"name": "DESSERTS14.pdf", "type": "desserts"}, 
#     {"name": "SAKE COCKTAILS14.pdf", "type": "drinks"}, 
#     {"name": "WEB WINE LIST.pdf", "type": "wine"}
# ]

# from pdfminer.high_level import extract_text

# for menu in menus:
#     text = extract_text(menu["name"])
#     name = restaurant.lower() + "-" + menu["type"] + ".txt"

#     with open(name, "w") as f:
#         f.write(text)

### Thái So´n

In [22]:
resturant = ""
items = []

response = requests.get("https://www.yelp.com/menu/th%C3%A1i-son-new-york-3")
doc = BeautifulSoup(response.text)

for item in doc.find("div", class_="menu-sections"):
    if item.find("a") != -1 and item.find("a") != None:
        name = item.find("a").text
    if item.find("p") != -1 and item.find("p") != None:
        desc = item.find("p").text.replace("\n", "").strip()
    if item.find("li") != -1 and item.find("li") != None:
        price = item.find("li", class_="menu-item-price-amount").text.replace("\n", "").replace(" ", "")
        
        item = {
            "name": name,
            "desc": desc,
            "price": price
        }
        
        items.append(item)

In [23]:
df = pd.DataFrame(items)
df.to_csv("thai-so'n.csv", index=False)

df["restaurant"] = "Thái So´n"
df["type"] = "TKTK"
df

,name,desc,price,restaurant,type
0,Summer Roll Shrimp Vegetable,with rice paper or vegetarian summer roll. 2 roll,$3.50,Thái So´n,TKTK
1,Combination Extra Big Bowl,"rice noodle beef soup six differences brisket,...",$5.75,Thái So´n,TKTK
2,Sauteed Frog Legs,with chili and lemongrass sauce,$9.25,Thái So´n,TKTK
3,Beef And Vegetable Chow Fun,with chili and lemongrass sauce,$9.95,Thái So´n,TKTK
4,Grilled Pork With Lettuce On Rice Vermicelli,with chili and lemongrass sauce,$5.50,Thái So´n,TKTK
5,"Shrimp With Little Salt, Green Pepper, Onions ...",with lemongrass and hot pepper,$8.50,Thái So´n,TKTK
6,Beef With Broccoli In Oyster Sauce,with lemongrass and hot pepper,$9.00,Thái So´n,TKTK
7,Barbecued Pork Chop,with lemongrass and hot pepper,$8.50,Thái So´n,TKTK
8,Chicken With Broccoli,with lemongrass and hot pepper,$8.50,Thái So´n,TKTK
9,Sauteed Chinese Mushroom With Vegetable,with lemongrass and hot pepper,$6.95,Thái So´n,TKTK


### Đi Ăn Đi

In [24]:
!pip install pillow-avif-plugin

You should consider upgrading via the '/Users/onlyandrewn/.pyenv/versions/3.10.3/bin/python3.10 -m pip install --upgrade pip' command.


In [25]:
from PIL import Image
import pillow_avif

In [26]:
restaurant = "di-an-di"

images = [
    {"old_name": "DAD2202_winter-menu-2022_v7-220121_1.avif", "new_name": "di-an-di-dinner.jpg", "type": "dinner"}, 
    {"old_name": "Drink_Menu_PG1.avif", "new_name": "di-an-di-drinks.jpg", "type": "drinks"}, 
    {"old_name": "DAD_LUNCH_MENU_-_012522.avif", "new_name": "di-an-di-lunch.jpg", "type": "lunch"}
]

for image in images:
    img = Image.open(image["old_name"])
    img.save(image["new_name"])

In [27]:
import pytesseract

In [28]:
## NOTE - MANUALLY CONVERTED AVIF FORMAT TO PDF

for image in images:
    text = pytesseract.image_to_string(image["new_name"])
    name = restaurant.lower() + "-" + image["type"] + ".txt"
    
    with open(name, "w") as f:
        f.write(text)

In [29]:
# test = []

# test.append("")

# df = pd.DataFrame("di-an-di-dinner.txt")
# df

### Van Đa

In [30]:
restaurant = "Van Đa"
items = []

response = requests.get("https://www.vanda.nyc/fullmenu")
doc = BeautifulSoup(response.text)
# doc

for item in doc.find_all("div", class_="sqs-block-content"):
    if item.find("div", class_="image-title sqs-dynamic-text"):
        name = item.find("div", class_="image-title sqs-dynamic-text").find("p").text
        
    if item.find("div", class_="image-subtitle sqs-dynamic-text"):
        desc = item.find("div", class_="image-subtitle sqs-dynamic-text").find("p").text

        item_obj = {
            "restaurant": restaurant,
            "name": name,
            "desc": desc
        }
        
        items.append(item_obj)

In [31]:
df = pd.DataFrame(items)

df["price"] = df.name.str.extract("(\d+)")
df["type"] = ""

# SELECT ROWS 0-3 AND COLUMN 3
df.iloc[0:4, 4] = "Street Food"
df.iloc[4:8, 4] = "Dumplings"
df.iloc[8:11, 4] = "Hanoi"
df.iloc[11:13, 4] = "Saigon"
df.iloc[14, 4] = "Che"
df.iloc[16:22, 4] = "Cocktail"
df.iloc[22, 4] = "Mocktail"

df = df.drop(df.index[23:36])

# df["ingredients"] = df["desc"]

df["ingredients"] = df["desc"]

In [32]:
# PROBLEM - TRYING TO SPLIT INGREDIENTS BY COMMA, .REPLACE NOT WORKING

# df["ingredients"].replace("mango", "ASDASDAS")
df.to_csv("van-da.csv", index=False)

### Bolero

In [33]:
restaurant = "Bolero"

# items_beverage = []
# items_summer = []
# items_dessert = []

# links = [
#     {"link": "https://bolero.nyc/beverage/", "type": "beverage"}, 
#     {"link": "https://bolero.nyc/menu-summer/", "type": "summer"}, 
#     {"link": "https://bolero.nyc/dessert/", "type": "dessert"}
# ]

In [34]:
#### PROBLEM - IT'S LOOPING OVER ALL THE ITEMS MULTIPLE TIMES

In [35]:
response = requests.get("https://bolero.nyc/beverage/")
doc = BeautifulSoup(response.text)

for item in doc.find("div", class_="bc-rich-text public-DraftEditor-content").select("p"):    
    items_beverage.append(item.text)
    
df = pd.DataFrame(items_beverage)
df.to_csv("bolero_beverage.csv", index=False)

In [37]:
response = requests.get("https://bolero.nyc/menu-summer/")
doc = BeautifulSoup(response.text)

for item in doc.find_all("div", class_="bc-text-wrapper"):
    for i in item.find("div", class_="component-inner").find_all("p"):    
        items_summer.append(i.text)

df = pd.DataFrame(items_summer)
df.to_csv("bolero_summer.csv", index=False)

In [39]:
response = requests.get("https://bolero.nyc/dessert/")
doc = BeautifulSoup(response.text)

for item in doc.find_all("div", class_="bc-text-wrapper"):
    for i in item.find("div", class_="component-inner").find_all("p"):    
        items_dessert.append(i.text)
        
df = pd.DataFrame(items_dessert)
df.to_csv("bolero_dessert.csv", index=False)

## MERGE ALL RESTAURANT MENUS TOGETHER

In [41]:
# import glob
# import os

In [47]:
# DOWNLOAD COLUMN IN DATAFRAME AS TXT FILE
# with open("images.txt", "w") as f_out:
#     f_out.write("\n".join(df["image"]))

In [ ]:
# TODO - DOWNLOADS THE FILE IN EACH OF THE LINKS, BUT NOT IN THE PROPER .JPG OR WITH THE SLUG
# !wget -i images.txt

In [ ]:
# Download a folder of pictures associated with each restaurant / dish
# ".csv", "Thái So´n.csv", "Đi Ăn Đi.csv", "Van Đa.csv", "Bolero.csv"

# restaurants = {
#     "https://www.yelp.com/biz/falansai-brooklyn-2",
#     "https://www.yelp.com/biz_photos/th%C3%A1i-son-new-york-3",
#     "https://www.yelp.com/biz_photos/di-an-di-brooklyn-2",
#     "https://www.yelp.com/biz_photos/van-%C4%91a-new-york-3",
#     "https://www.yelp.com/biz_photos/bolero-brooklyn"
# }

In [46]:
menus = ["clean-falansai.csv", "clean-thai-so'n.csv", "clean-di-an-di.csv", "clean-van-da.csv", "clean-bolero.csv"]

combined = pd.concat([pd.read_csv(menu) for menu in menus])
combined.to_csv("combined.csv", index=False)

## QUESTIONS

In [86]:
# What is the most common dish? What dishes are staples?
# How similar are the restaurants?
# What are the most unique dishes?

# TODO - SPLIT EACH ROW BY COMMA

df = pd.DataFrame(combined)
df = df.dropna()

for item in df["ingredients"]:
    print(item.lower())

oysters, clams, shrimp, chef’s crudo
kohlrabi mignonette, oysters
tilefish, gulf shrimp, fermented chili, avocado, sobre masa tostadas
guacamole, lemon, vermicelli, tuna
pickles
pork, mushrooms, lettuce, fish sauce
duck
vermicelli, egg, lemongrass, lamb
pho broth, squash, egg, cabbage
tofu, mushroom, rice, curry
vietnamese mortadella, chinese sausage, egg, onion, rice
pork shoulder, salsa verde, herbs, egg
pea tendrils, tamarind, lamb
red curry, cabbage, mackerel
duck, pork sausage, smoked fish, fried noodles
pineapple, soda
lemon, soda
mango, kombucha
blueberry, ginger, tonic
shrimp, vegetable, rice paper
pork,vegetable, rice paper
shrimp, green papaya, peanuts
shrimp, beef, chicken, vegetable, peanuts
shrimp, pork
pork, vegetable
squid
mussel, curry
crab
shrimp, sweet and sour sauce
frog legs, butter
shrimp, vegetable
fish, vegetable
fish, vegetable
chicken, vegetable
chicken, vegetable
vegetable, bean curd
vegetable, bean curd
seafood,vegetable,soup
seafood,vegetable,soup
rice noodl

In [85]:
df[df["restaurant"] == "bolero"].sort_values(by="dish", ascending=False)

,restaurant,dish,ingredients
40,bolero,“BÁNH MÌ (OPEN FACED VIETNAMESE SANDWICH),"chicken liver pate, mortadella, pork, sriracha..."
3,bolero,‘DEM APPLES,"bourbon, calvados, sherry, vermouth bianco"
32,bolero,​CÀ PHÊ PHIN ​(HOT PHIN FILTER DRIP COFFEE),"coffee, condensed milk"
25,bolero,​CÀ PHÊ PHIN ​(HOT PHIN FILTER DRIP COFFEE),"coffee, condensed milk"
39,bolero,XÔI CHIÊN (FRIED STICKY RICE ARANCINI),"mushroom, cheese"
20,bolero,TRÀ ĐÁ // VIETNAMESE ICE TEA,"vietnamese white tea, rock sugar"
6,bolero,THE YUZUAL,"mezcal, pineapple, yuzu, dill"
15,bolero,THE BOLERO EGG CHALLENGE,"beer, rice whiskey, egg, coriander, salt, pepper"
5,bolero,RUM & CUKE,"rum, cucumber, basil, pandan, lime, soda"
7,bolero,PINK + WHITE​ (SOLD OUT),"vodka, grapefruit, white peppercorn, coriander..."


In [154]:
df = pd.read_csv("ingredients.csv")

df["ingredients"] = df["ingredients"].replace("shrimp", "Shrimp") \
    .replace("pork", "Pork") \
    .replace("beef", "Beef") \
    .replace("Rice noodles", "Noodles") \
    .replace("noodle", "Noodles") \
    .replace("noodles", "Noodles") \
    .replace("fried noodles", "Noodles") \
    .replace("lemongrass", "Lemongrass") \
    .replace("egg", "Egg") \
    .replace("fish sauce", "Fish sauce") \
    .replace("cucumber", "Cucumber") \
    .replace("cucumbers", "Cucumber") \
    .replace("Vermicelli", "Noodles") \
    .replace("lime", "Lime") \
    .replace("Vegetable", "Vegetables") \
    .replace("Vegetable", "Vegetables") \
    .replace("tofu", "Tofu") \
    .replace("lemon", "Lemon") \
    .replace("shallots", "Shallots") \
    .replace("scallion", "Scallions") \
    .replace("mushrooms", "Mushrooms") \
    .replace("chili", "Chili") \
    .replace("coffee", "Coffee") \
    .replace("rice paper", "Rice paper") \
    .replace("nuoc cham", "Fish sauce")

df.drop(df[df['ingredients'] == "Vegetables"].index, inplace = True)
df.drop(df[df['ingredients'] == "herbs"].index, inplace = True)
df.drop(df[df['ingredients'] == "Coffee"].index, inplace = True)

df.groupby("ingredients").value_counts().sort_values(ascending=False).head(10)

ingredients
Noodles       42
Beef          39
Shrimp        36
Pork          34
Lemongrass    22
Chicken       22
Rice          19
Egg           17
Fish sauce    15
Chili         14
dtype: int64

In [163]:
# pd.set_option('display.max_rows', 450)

# df.groupby("ingredients").value_counts().sort_values(ascending=False).head(400)
df.groupby("ingredients").value_counts().sum()

960